In [50]:
# USER OPTIONS
# define tokenizer name - should be one among the keys in the cell below
tokenizer_name = 'ChordSymbolTokenizer' # or any other name from the keys in tokenizers dictionary
# tokenizer_name = 'RootTypeTokenizer'
# tokenizer_name = 'PitchClassTokenizer'
# folder to xmls
val_dir = '/media/maindisk/maximos/data/hooktheory_test'
# val_dir = '/media/maindisk/maximos/data/gjt_melodies/Library_melodies'
# val_dir = '/media/datadisk/datasets/gjt_melodies/Library_melodies'
# val_dir = '/media/maximos/9C33-6BBD/data/gjt_melodies/Library_melodies'

# define batch size depending on GPU availability / status
batchsize = 16
# select device name - could be 'cpu', 'cuda', 'coda:0', 'cuda:1'...
device_name = 'cuda'

In [51]:
import pickle
with open('data/idioms_mir_quick_reference.pickle', 'rb') as f:
    idioms = pickle.load(f)

In [52]:
bc_major = idioms['BachChorales_[0 2 4 5 7 9 11]']
bc_minor = idioms['BachChorales_[0 2 3 5 7 8 10]']
jazz_maj = idioms['Jazz_[0 2 4 5 7 9 11]']
jazz_min = idioms['Jazz_[0 2 3 5 7 9 10]']
organum = idioms['organum_[0 2 3 5 7 8 10]']

In [53]:
from data_utils import SeparatedMelHarmMarkovDataset
import os
import numpy as np
from harmony_tokenizers_m21 import ChordSymbolTokenizer, RootTypeTokenizer, \
    PitchClassTokenizer, RootPCTokenizer, GCTRootPCTokenizer, \
    GCTSymbolTokenizer, GCTRootTypeTokenizer, MelodyPitchTokenizer, \
    MergedMelHarmTokenizer
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader
from transformers import BartForConditionalGeneration, BartConfig, DataCollatorForSeq2Seq
from tqdm import tqdm
from models import TransGraphVAE
import csv
from transformers import LogitsProcessor, StoppingCriteria, StoppingCriteriaList

In [54]:
tokenizers = {
    'ChordSymbolTokenizer': ChordSymbolTokenizer,
    'RootTypeTokenizer': RootTypeTokenizer,
    'PitchClassTokenizer': PitchClassTokenizer,
    'RootPCTokenizer': RootPCTokenizer,
    'GCTRootPCTokenizer': GCTRootPCTokenizer,
    'GCTSymbolTokenizer': GCTSymbolTokenizer,
    'GCTRootTypeTokenizer': GCTRootTypeTokenizer
}

In [55]:
melody_tokenizer = MelodyPitchTokenizer.from_pretrained('saved_tokenizers/MelodyPitchTokenizer')
harmony_tokenizer = tokenizers[tokenizer_name].from_pretrained('saved_tokenizers/' + tokenizer_name)

tokenizer = MergedMelHarmTokenizer(melody_tokenizer, harmony_tokenizer)

bart_path = 'saved_models/bart/' + tokenizer_name + '/' + tokenizer_name + '.pt'

In [56]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [57]:
bart_config = BartConfig(
    vocab_size=len(tokenizer.vocab),
    pad_token_id=tokenizer.pad_token_id,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    decoder_start_token_id=tokenizer.bos_token_id,
    forced_eos_token_id=tokenizer.eos_token_id,
    max_position_embeddings=512,
    encoder_layers=8,
    encoder_attention_heads=8,
    encoder_ffn_dim=512,
    decoder_layers=8,
    decoder_attention_heads=8,
    decoder_ffn_dim=512,
    d_model=512,
    encoder_layerdrop=0.3,
    decoder_layerdrop=0.3,
    dropout=0.3
)

bart = BartForConditionalGeneration(bart_config)

if torch.cuda.is_available():
    checkpoint = torch.load(bart_path, weights_only=True)
else:
    checkpoint = torch.load(bart_path, map_location="cpu", weights_only=True)
bart.load_state_dict(checkpoint)

<All keys matched successfully>

In [58]:
enc1 = bart.model.encoder
enc2 = bart.get_encoder()
dec1 = bart.model.decoder
dec2 = bart.get_decoder()

In [59]:
dec1 == dec2

True

In [60]:
test_dir = '/mnt/ssd2/maximos/data/hooktheory_test'
test_dataset = SeparatedMelHarmMarkovDataset(test_dir, tokenizer, max_length=512, num_bars=64)

# Data collator for BART
def create_data_collator(tokenizer, model):
    return DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, padding=True)
# end create_data_collator

In [61]:
collator = create_data_collator(tokenizer, model=bart)
valloader = DataLoader(test_dataset, batch_size=1, shuffle=True, collate_fn=collator)

In [62]:
model_path = 'saved_models/bart_cvae/' + tokenizer_name + '/' + tokenizer_name + '.pt'

config = {
    'hidden_dim_LSTM': 1024,
    'hidden_dim_GNN': 1024,
    'latent_dim': 1024,
    'condition_dim': 1024,
    'use_attention': False
}

model = TransGraphVAE(transformer=bart, device=device, tokenizer=tokenizer, **config)

if torch.cuda.is_available():
    checkpoint = torch.load(model_path, weights_only=True)
else:
    checkpoint = torch.load(model_path, map_location="cpu", weights_only=True)
model.load_state_dict(checkpoint)

model.to(device)
model.eval()

TransGraphVAE(
  (transformer): BartForConditionalGeneration(
    (model): BartModel(
      (shared): BartScaledWordEmbedding(545, 512, padding_idx=1)
      (encoder): BartEncoder(
        (embed_tokens): BartScaledWordEmbedding(545, 512, padding_idx=1)
        (embed_positions): BartLearnedPositionalEmbedding(514, 512)
        (layers): ModuleList(
          (0-7): 8 x BartEncoderLayer(
            (self_attn): BartSdpaAttention(
              (k_proj): Linear(in_features=512, out_features=512, bias=True)
              (v_proj): Linear(in_features=512, out_features=512, bias=True)
              (q_proj): Linear(in_features=512, out_features=512, bias=True)
              (out_proj): Linear(in_features=512, out_features=512, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=512, out_features=512, bias=True)
            (fc2): Linear(in_feat

In [63]:
batch = next(iter(valloader))

In [64]:
with torch.no_grad():
    input_ids = batch['input_ids'].to(device)
    # transitions = batch['transitions'].to(device)
    # transitions = torch.tensor(bc_major).reshape(1,bc_major.shape[0], bc_major.shape[1]).to(device)
    # transitions = torch.tensor(bc_minor).reshape(1,bc_minor.shape[0], bc_minor.shape[1]).to(device)
    # transitions = torch.tensor(organum).reshape(1,organum.shape[0], organum.shape[1]).to(device)
    # transitions = torch.tensor(jazz_maj).reshape(1,jazz_maj.shape[0], jazz_maj.shape[1]).to(device)
    transitions = torch.tensor(jazz_min).reshape(1,jazz_min.shape[0], jazz_min.shape[1]).to(device)
    # attention_mask = batch['attention_mask'].to(device)
    attention_mask = None
    output_tokens = []
    output_recon_tokens = []
    num_bars = (input_ids == tokenizer.vocab['<bar>']).sum(dim=1).reshape(input_ids.shape[0],-1)
    print(input_ids)
    print(transitions)
    print(num_bars.shape)
    outputs = model(input_ids, transitions, encoder_attention=attention_mask, generate_max_tokens=500, num_bars=num_bars, temperature=1.0)
    for i in outputs['generated_ids'][0]:
        output_tokens.append( tokenizer.ids_to_tokens[ int(i) ].replace(' ','x') )
    for i in outputs['generated_recon_ids'][0]:
        output_recon_tokens.append( tokenizer.ids_to_tokens[ int(i) ].replace(' ','x') )

tensor([[  2,   6, 180,  95,  50,  99,  51, 103,  58, 107,  50, 109,  51, 111,
          50, 115,  51, 119,  58, 123,  50, 125,  51,   6,  95,  50,  99,  51,
         103,  58, 107,  50, 109,  51, 111,  50, 115,  51, 119,  58, 123,  50,
         125,  51,   6,  95,  49,  99,  50, 103,  57, 107,  49, 109,  50, 111,
          49, 115,  50, 119,  57, 123,  49, 125,  50,   6,  95,  49,  99,  50,
         103,  57, 107,  49, 109,  50, 111,  49, 115,  50, 119,  57, 123,  49,
         125,  50]], device='cuda:0')
tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]], device='cuda:0',
       dtype=torch.float64)
torch.Size([1, 1])
recon generation
bars_left: tensor([[4]], device='cuda:0')
bars_left: tensor([[4]], device='cuda:0')
bars_left: tensor([[3]], device='cuda:0')
bars_left: tensor([

In [65]:
print(output_tokens)

['<s>', '<h>', '<bar>', 'position_0x00', 'A:min7', 'position_1x50', 'G:maj', '<bar>', 'position_0x00', 'F:maj7', 'position_1x50', 'C:maj', '<bar>', 'position_0x00', 'A:min7', 'position_1x50', 'G:maj', '<bar>', 'position_0x00', 'F:maj7', 'position_1x50', 'C:maj', '</s>']


In [66]:
print(output_recon_tokens)

['<s>', '<h>', '<bar>', 'position_0x00', 'A:min', 'position_1x00', 'A:min', 'position_2x00', 'A:min', 'position_3x00', 'A:min', '<bar>', 'position_0x00', 'A:min', 'position_1x00', 'A:min', 'position_2x00', 'A:min', 'position_3x00', 'A:min', '<bar>', 'position_0x00', 'G#:min', 'position_1x00', 'G#:min', 'position_2x00', 'G#:min', 'position_3x00', 'G#:min', '<bar>', 'A:dim', 'position_1x00', 'G#:min', 'position_2x00', 'G#:min', 'position_3x00', 'G#:min', 'position_1x00', 'G#:min', 'position_2x00', 'G#:min', 'position_3x00', 'G#:min', 'position_1x00', 'G#:min', 'position_2x00', 'G#:min', 'position_3x00', 'G#:min', '</s>']


In [67]:
input_ids = batch['input_ids'].to(device)
print(input_ids)
input_tokens = []
for i in input_ids[0]:
    input_tokens.append( tokenizer.ids_to_tokens[ int(i) ].replace(' ','x') )
print(input_tokens)

tensor([[  2,   6, 180,  95,  50,  99,  51, 103,  58, 107,  50, 109,  51, 111,
          50, 115,  51, 119,  58, 123,  50, 125,  51,   6,  95,  50,  99,  51,
         103,  58, 107,  50, 109,  51, 111,  50, 115,  51, 119,  58, 123,  50,
         125,  51,   6,  95,  49,  99,  50, 103,  57, 107,  49, 109,  50, 111,
          49, 115,  50, 119,  57, 123,  49, 125,  50,   6,  95,  49,  99,  50,
         103,  57, 107,  49, 109,  50, 111,  49, 115,  50, 119,  57, 123,  49,
         125,  50]], device='cuda:0')
['<s>', '<bar>', 'ts_4x4', 'position_0x00', 'P:64', 'position_0x50', 'P:65', 'position_1x00', 'P:72', 'position_1x50', 'P:64', 'position_1x75', 'P:65', 'position_2x00', 'P:64', 'position_2x50', 'P:65', 'position_3x00', 'P:72', 'position_3x50', 'P:64', 'position_3x75', 'P:65', '<bar>', 'position_0x00', 'P:64', 'position_0x50', 'P:65', 'position_1x00', 'P:72', 'position_1x50', 'P:64', 'position_1x75', 'P:65', 'position_2x00', 'P:64', 'position_2x50', 'P:65', 'position_3x00', 'P:72', 'p

In [68]:
class ExactTokenCountLogitsProcessor(LogitsProcessor):
    def __init__(self, token_id, max_count):
        self.token_id = token_id
        self.max_count = max_count

    def __call__(self, input_ids, scores):
        token_count = (input_ids == self.token_id).sum().item()
        if token_count >= self.max_count:
            scores[:, self.token_id] = -float("inf")  # Mask the token
        return scores

class ExactTokenCountStoppingCriteria(StoppingCriteria):
    def __init__(self, token_id, max_count):
        self.token_id = token_id
        self.max_count = max_count

    def __call__(self, input_ids, scores, **kwargs):
        token_count = (input_ids == self.token_id).sum().item()
        return token_count >= self.max_count  # Stop when the count is reached


In [69]:
# Define the token ID and the exact count you want
token_id = 200
exact_count = 10

bart_outputs = bart.generate(
    input_ids=input_ids,
    # attention_mask=batch['attention_mask'][bi],
    bos_token_id=tokenizer.vocab[tokenizer.harmony_tokenizer.start_harmony_token],
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=500,
    do_sample=True,
    temperature=1.0,
    # logits_processor=[ExactTokenCountLogitsProcessor(token_id, exact_count)],
    # stopping_criteria=StoppingCriteriaList([ExactTokenCountStoppingCriteria(token_id, exact_count)])
)

In [70]:
print(bart_outputs)
bart_only_outputs = []
for i in bart_outputs[0]:
    bart_only_outputs.append( tokenizer.ids_to_tokens[ int(i) ].replace(' ','x') )
print(bart_only_outputs)

tensor([[  2, 196,   6,  95, 459, 103, 459, 109, 459, 111, 459, 115, 459, 119,
         459,   6, 103, 459, 109, 459, 111, 459, 115, 459, 119, 459, 125, 459,
           6,  95, 430, 103, 430, 109, 313, 109, 319, 111, 313, 115, 430, 119,
         313, 125, 430,   6, 103, 430, 109, 430, 111, 256, 113, 342,   3]],
       device='cuda:0')
['<s>', '<h>', '<bar>', 'position_0x00', 'A:min', 'position_1x00', 'A:min', 'position_1x75', 'A:min', 'position_2x00', 'A:min', 'position_2x50', 'A:min', 'position_3x00', 'A:min', '<bar>', 'position_1x00', 'A:min', 'position_1x75', 'A:min', 'position_2x00', 'A:min', 'position_2x50', 'A:min', 'position_3x00', 'A:min', 'position_3x75', 'A:min', '<bar>', 'position_0x00', 'G#:min', 'position_1x00', 'G#:min', 'position_1x75', 'E:maj', 'position_1x75', 'E:7', 'position_2x00', 'E:maj', 'position_2x50', 'G#:min', 'position_3x00', 'E:maj', 'position_3x75', 'G#:min', '<bar>', 'position_1x00', 'G#:min', 'position_1x75', 'G#:min', 'position_2x00', 'D:min', 'position_

In [71]:
print(bart_only_outputs)
print(output_tokens)
print(output_recon_tokens)

['<s>', '<h>', '<bar>', 'position_0x00', 'A:min', 'position_1x00', 'A:min', 'position_1x75', 'A:min', 'position_2x00', 'A:min', 'position_2x50', 'A:min', 'position_3x00', 'A:min', '<bar>', 'position_1x00', 'A:min', 'position_1x75', 'A:min', 'position_2x00', 'A:min', 'position_2x50', 'A:min', 'position_3x00', 'A:min', 'position_3x75', 'A:min', '<bar>', 'position_0x00', 'G#:min', 'position_1x00', 'G#:min', 'position_1x75', 'E:maj', 'position_1x75', 'E:7', 'position_2x00', 'E:maj', 'position_2x50', 'G#:min', 'position_3x00', 'E:maj', 'position_3x75', 'G#:min', '<bar>', 'position_1x00', 'G#:min', 'position_1x75', 'G#:min', 'position_2x00', 'D:min', 'position_2x25', 'F:maj', '</s>']
['<s>', '<h>', '<bar>', 'position_0x00', 'A:min7', 'position_1x50', 'G:maj', '<bar>', 'position_0x00', 'F:maj7', 'position_1x50', 'C:maj', '<bar>', 'position_0x00', 'A:min7', 'position_1x50', 'G:maj', '<bar>', 'position_0x00', 'F:maj7', 'position_1x50', 'C:maj', '</s>']
['<s>', '<h>', '<bar>', 'position_0x00', '

In [72]:
tokenizer.decode( input_tokens + bart_only_outputs[1:], output_format='file', output_path='examples/bart.mxl' )
tokenizer.decode( input_tokens + output_tokens[1:], output_format='file', output_path='examples/encdec.mxl' )
tokenizer.decode( input_tokens + output_recon_tokens[1:], output_format='file', output_path='examples/recon.mxl' )

Saved as examples/bart.mxl
Saved as examples/encdec.mxl
Saved as examples/recon.mxl


In [73]:
print( (outputs['generated_markov']-transitions.to('cpu')).pow(2).sum().sqrt() )
print( (outputs['recon_markov']-transitions.to('cpu')).pow(2).sum().sqrt() )

tensor(15.6729, dtype=torch.float64)
tensor(11.3859, dtype=torch.float64)
